# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [ ]:
!pip install turicreate
from google.colab import drive
drive.mount('/content/gdrive')
import turicreate

     |████████████████████████████████| 91.9MB 45kB/s 
     |████████████████████████████████| 2.1MB 33.5MB/s 
     |████████████████████████████████| 86.3MB 36kB/s 
     |████████████████████████████████| 20.2MB 1.2MB/s 
     |████████████████████████████████| 3.4MB 36.7MB/s 
     |████████████████████████████████| 3.8MB 23.0MB/s 
     |████████████████████████████████| 450kB 58.7MB/s 
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0
  Found existing installation: google-auth-oauthlib 0.4.0
    Uninstalling google-auth-oauthlib-0.4.0:
      Successfully uninstalled google-auth-oauthlib-0.4.0
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensor

Mounted at /content/gdrive


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [ ]:
import math
from math import log
sales = turicreate.SFrame('/content/gdrive/My Drive/Turicreate/Regression/Week2/home_data.sframe')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [18]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'] * train_data['bedrooms']
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
train_data['log_sqft_living'] = train_data.apply(lambda x: math.log(x['sqft_living']))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['bedrooms_squared'] = test_data['bedrooms'] * test_data['bedrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: math.log(x))
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']
#print train_data
#print train_data['log_sqft_living']
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                    validation_set = None)
print test_data['bedrooms_squared'].mean()
print test_data['bed_bath_rooms'].mean()
print test_data['log_sqft_living'].mean()
print test_data['lat_plus_long'].mean()

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.017539     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

12.4466777016
7.50390163159
7.55027467965
-74.6533349722


Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [ ]:
features_model1 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model1 = turicreate.linear_regression.create(train_data, target = 'price', features = features_model1, validation_set = None)
features_model2 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat','long', 'bed_bath_rooms']
model2 = turicreate.linear_regression.create(train_data, target = 'price', features = features_model2, validation_set = None)
features_model3 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
model3 = turicreate.linear_regression.create(train_data, target = 'price', features = features_model3, validation_set = None)
example_weight_summary = example_model.coefficients
print example_weight_summary

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| 1         | 2        | 0.027302     | 3193229.177888     | 228200.043155                   |

| 2         | 3        | 0.049844     | 3193229.177873     | 228200.043155                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019264  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [ ]:
#model1
model2
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

Class                          : LinearRegression

Schema
------
Number of coefficients         : 7
Number of examples             : 17384
Number of feature columns      : 6
Number of unpacked features    : 6

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 2
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0484

Settings
--------
Residual sum of squares        : 9.61592067858e+14
Training RMSE                  : 235190.9354

Highest Positive Coefficients
-----------------------------
lat                            : 625433.8349
bed_bath_rooms                 : 26961.6249
sqft_living                    : 304.4493

Lowest Negative Coefficients
----------------------------
(intercept)                    : -54410676.1118
long                           : -203958.6029
bedroom

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)

    # Then compute the residuals/errors
    residuals = predictions - outcome

    # Then square and add them up
    residuals_squared = residuals * residuals
    RSS = sum(residuals_squared)

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

12.4466777016


In [ ]:
# create the remaining 3 features in both TEST and TRAIN data



* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)


In [ ]:
# Examine/extract each model's coefficients:


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_1_train_RSS = get_residual_sum_of_squares(model1, train_data, train_data['price'])
model_2_train_RSS = get_residual_sum_of_squares(model2, train_data, train_data['price'])
model_3_train_RSS = get_residual_sum_of_squares(model3, train_data, train_data['price'])
print model_1_train_RSS
print model_2_train_RSS
print model_3_train_RSS

9.71328233545e+14
9.61592067858e+14
9.05276314552e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model_1_test_RSS = get_residual_sum_of_squares(model1, test_data, test_data['price'])
model_2_test_RSS = get_residual_sum_of_squares(model2, test_data, test_data['price'])
model_3_test_RSS = get_residual_sum_of_squares(model3, test_data, test_data['price'])
print model_1_test_RSS
print model_2_test_RSS
print model_3_test_RSS

2.26568089093e+14
2.24368799994e+14
2.51829318963e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.